# Install Transformers Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel

# specify GPU
device = torch.device("cuda")

# Load Dataset

In [ ]:
df = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
np.random.seed(0)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
# nltk.download('wordnet')

In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@",'', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    text = re.sub(r'<URL>','',text)

    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    text = contractions.fix(text)
    # text = re.sub(r'\.+','',text)
    return text


In [ ]:
def fun(A,l):
  if len(A)<=l:
    return A
  else:
    return  A[-l::1]
    # return A[-1:-l-1:-1]
def fun1(A):
  return ''.join(A)

In [ ]:
l=2

# df['tweets']= df['response'] +'reply-'+ df['context'].apply(fun,args=[l]).apply(fun1)
# df1['tweets']=df1['response'] +'reply-'+ df1['context'].apply(fun,args=[l]).apply(fun1)

df['tweets']= 'reply-'+ df['response']# + ' context- ' + df['context'].apply(fun,args=[l]).apply(fun1)
df1['tweets']= 'reply-'+ df1['response']# +  ' context- ' +  df1['context'].apply(fun,args=[l]).apply(fun1)


# df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) + df['response']
# df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) + df1['response']

df['tweets']= df['tweets'].apply(preprocess_text) 
df1['tweets']=df1['tweets'].apply(preprocess_text)
# df['tweets']=df['tweets']+'reply-'+ df['response'].apply(preprocess_text)
# df1['tweets']=df1['tweets']+'reply-'+ df1['response'].apply(preprocess_text)

In [ ]:
# check class distribution
df['labels'].value_counts(normalize = True)

In [ ]:
df.loc[2628,'tweets']

In [ ]:
# df["tweets"]=df['response']
# df1["tweets"]=df1['response']

In [ ]:
df.shape

In [ ]:
# check class distribution
df['labels'].value_counts(normalize = True)

# Split train dataset into train, validation and test sets

In [ ]:

X_s = df['tweets'].values
y_s = df['labels'].values

Xt_s = df1['tweets'].values
yt_s = df1['labels'].values

train_text, temp_text, train_labels, temp_labels = X_s,Xt_s,y_s,yt_s

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
val_text, test_text, val_labels, test_labels=temp_text, temp_text, temp_labels, temp_labels

# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)


In [ ]:
# pip install -U sentence-transformers

# Import BERT Model and BERT Tokenizer

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel
bert = RobertaModel.from_pretrained('roberta-large')

# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

In [ ]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [ ]:
# output
print(sent_id)

# Tokenization

In [ ]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in train_text]
pd.Series(seq_len).hist(bins = 30)

In [ ]:
max_seq_len = 120

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# Convert Integer Sequences to Tensors

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True
# print(bert)

# Define Model Architecture

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(1024,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id,mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# # optimizer from hugging face transformers
from transformers import AdamW

# # define the optimizer
# decayRate = 0.99
optimizer = AdamW(model.parameters(), lr = 2e-5)
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [ ]:
# from scheduler import CyclicCosineDecayLR
# import torch.optim as optim
# # optimizer = optim.SGD(model.parameters(), lr=2e-3)
# my_lr_scheduler = CyclicCosineDecayLR(optimizer, 
#                                 init_decay_epochs=3,
#                                 min_decay_lr=5e-6,
#                                 restart_interval=6,
#                                 restart_interval_multiplier=1.2,
#                                 restart_lr=5e-5)

# Find Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

In [ ]:
from tqdm.auto import tqdm

# Fine-Tune BERT

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in tqdm(enumerate(train_dataloader),total=4400//batch_size+1):
    # progress update after every 50 batches.
    # if step % 50 == 0 and not step == 0:
    #   print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    # my_lr_scheduler.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  # for step,batch in enumerate(val_dataloader):
  for step,batch in tqdm(enumerate(val_dataloader),total=1800//batch_size+1):
    # Progress update every 50 batches.
    # if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      # print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Start Model Training

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
epochs=30
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    for param_group in optimizer.param_groups:
        print('learning_rate- ',param_group['lr'])
    #train model
    train_loss,_ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    # if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'saved_weights.pt')


    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    pred=[]
    with torch.no_grad():
      for i in tqdm(range(0,90)):
        preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
        preds = preds.detach().cpu().numpy()
        pred.append(preds)
    pred1=np.array(pred)
    pred2=pred1.reshape((1800,2))
    p=pd.DataFrame(pred2)
    p.to_csv('71_Rresponse.csv')
    pred3 = np.argmax(pred2, axis = 1)
    print(classification_report(test_y, pred3))
    print(accuracy_score(test_y, pred3))

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

# Load Saved Model

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

# Get Predictions for Test Data

In [ ]:
pred=[]
with torch.no_grad():
  for i in range(0,90):
    preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
    preds = preds.detach().cpu().numpy()
    pred.append(preds)
# print(np.array(pred).shape)

In [ ]:
pred1=np.array(pred)

In [ ]:
pred2=pred1.reshape((1800,2))
# pred2.shape

In [ ]:
p=pd.DataFrame(pred2)
p.to_csv('71_Rresponse.csv')

In [ ]:
pred3 = np.argmax(pred2, axis = 1)
# pred3 

In [ ]:
print(classification_report(test_y, pred3))